<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/PH_TrainTestSplit_Ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare whether you are on Colab or local
colab = True

In [4]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"

  #install required packages
  !pip install duckdb
  #!pip install -U sentence-transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached duckdb-0.4.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15.7 MB)


In [5]:
# Import sentence_transformers 
# from sentence_transformers import SentenceTransformer, util

In [6]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import duckdb
import sqlalchemy
#%load_ext sql

# Train-validation-test split the ontology

### Load data

#### Ontology + synonyms

In [7]:
if colab==True:
  ontology_syn = pd.read_csv(data_path+"/Synonyms in Ontology/isontology.csv", index_col=0)
else:
  ontology_syn = pd.read_csv("../Data/SynonymsinOntology/isontology.csv", index_col=0)
ontology_syn.shape

(397942, 4)

In [10]:
ontology_syn.head()

,entity_id,category,label,synonym
0,IS topic,domain specific entity,TOPIC,IS topic
1,IS topic,domain specific entity,TOPIC,IS topics
2,IS sourcing,domain specific entity,TOPIC,IS / IT sourcing
3,IS sourcing,domain specific entity,TOPIC,tool sourcing
4,IS sourcing,domain specific entity,TOPIC,software sourcings


In [11]:
## Check number of synonyms per entity_id
#ent_sy_group = ontology_syn.groupby("entity_id")[["synonym"]].nunique().sort_values(by="synonym", ascending=False)

In [12]:
# # Downsample entities with more than 100 synonyms to a 100 synoyms (but always include "main entity_id" in subsample)
# idx = ent_sy_group[ent_sy_group["synonym"]>100].index.tolist()

# dfs = []
# for entity in idx:
#   t1 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym==entity)]
#   t2 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym!=entity)].sample(99)
#   dfs.append(pd.concat([t1,t2]))
# dfs.append(ontology_syn.loc[~ontology_syn["entity_id"].isin(idx)]) # Append dataframe for all entities with less then 100 synonyms to list

# # Combine all dataframes 
# ontology_syn_ds = pd.concat(dfs).reset_index(drop=True)

In [13]:
#Show synonyms in ontology which appear in more than 1 entity
#ontology_syn_ds[ontology_syn_ds.synonym.duplicated(keep=False)].sort_values(by="synonym")

In [14]:
#List of unique entities from ontology with synonyms with downsampling (apparently there are some duplicates in there?!)
#entities_syn_unique_ds = ontology_syn_ds.synonym.unique().tolist()
#ontology_syn_ds_unique = ontology_syn_ds.drop_duplicates(subset="synonym")

#### Ontology.json (only structure of ontology with "main" entity ids (without synonmys)

In [33]:
if colab==True:
  ontology_json = pd.read_json(data_path+"/Ontology/isontology.json")#, orient="values")#, index_col=0)

In [34]:
ontology_json

,root
0,{'IS topic': [{'IS sourcing': [{'outsourcing':...
1,{'research paradigm': [{'positivism': ['logica...
2,"{'model': ['taxonomy', 'framework', {'research..."
3,"{'economic sector': ['private household', {'go..."
4,{'company type': [{'small and mid-size enterpr...
5,{'participant': [{'individual participant': ['...
6,{'level of analysis': [{'macro level': [{'soci...
7,{'theory': [{'information systems theory': ['t...
8,{'data analysis method': [{'qualitative analys...
9,{'sampling': [{'non probabilistic sampling': [...


#### Entities (Sentence_id + tagged entity_id) by Rolands script

In [35]:
# Note that DBeaver must be closed, before you can run this command!
if colab==True:
  con = duckdb.connect(database=data_path+'/Extracted Information of Corpus/isrecon.duckdb', read_only=False)
else:
  con = duckdb.connect(database='../Data/ExtractedInformationofCorpus/isrecon.duckdb', read_only=False)

Lets load all the labeled main entities fro the duckdb

In [49]:
# Load first article as df
entities = con.execute("""
                       SELECT article_id, sentence_id, ent_id, level_1, level_2, level_3, level_4, level_5, level_6, level_7, level_8, level_9, level_10,
                       FROM entities e
                       """).fetchdf()

entities.shape

(1797740, 13)

In [72]:
# Main entities of first level of ontology
entities.level_1.unique()

array(['domain specific entity', 'theoretical entity',
       'methodological entity'], dtype=object)

In [73]:
# Main entities of 2nd level of ontology
entities.level_2.unique()

array(['IS technology', 'IS topic', 'economic sector', 'model',
       'study object', 'research method', 'data analysis method',
       'level of analysis', 'statistical metric', 'validity', 'theory',
       'sampling', 'research paradigm'], dtype=object)

In [81]:
# Distribution of entities from database accross first and second level
entities.groupby(["level_1", "level_2"])[["ent_id"]].aggregate(["count", "nunique"])

ent_id        
                                              count nunique
level_1                level_2                             
domain specific entity IS technology         463211     475
                       IS topic              461262     416
                       economic sector       102057     234
                       study object           61799      21
methodological entity  data analysis method   73002     292
                       research method       257820     272
                       sampling                6148      27
                       statistical metric     78673      87
                       validity               11357      25
theoretical entity     level of analysis      12064      13
                       model                 196827      31
                       research paradigm      13429      14
                       theory                 60091     223

## `Train-val-test` split of the existing ontology

In order to determine how well our model/embeddings are in identifing new entities or synonyms for existing entities we have to create a `train-val-test split` version of the existing ontology. Meaning we have to select some entities which will be removed from the ontology for training. 


- We will remove different nodes across all main topics and levels of the ontology to assess how our model is handling with missing nodes from different "levels"
- To determine which nodes to include we used the `ontology.json` file since here the hierarchy is already visually included, so it was easy to define some random nodes (and child nodes)

**Entities we selected to remove**:
- **domain specific entity**
  - IS topic
    - business process outsourcing
    - cybersecurity behavior
    - user interface
    - IT planning + all child nodes
  - IS technology
    - online marketing + all child nodes
    - video game
    - non volatile memory
  - Economic sector
    - forest industry + all child nodes
    - hedge fund
    - structural engineering + all child nodes
  - stody object
    - crowdsourcing of participant + all child nodes
    - startup
    - microenterprise
- **methodological entity**
  - research method
    - pilot experiment
    - completely randomized design
    - survey design problem + all child nodes
    - action research + all child nodes
    - delphi study
    - Markov model + all child nodes
    - database modeling
  - data analysis method
    - constant comparative analysis
    - ensemble learning + all child nodes
    - sentiment analysis + all child nodes
    - autoregressive moving average model
  - statistical metric
    - correlation direction + all child nodes
    - Chi square metric
    - confusion matrix
  - validity
    - formative validity
    - criterion validity + all child nodes
  - sampling
    - oversampling + all child nodes
    - quota sampling
- **theoretical entity**
  - level of analysis
    - micro level + all child nodes
    - country level
  - model
    - causal model
    - control variable
    - taxonomy
    - theoretical link + all child nodes
  - research paradigm
    - interpretivism
    - realism + all child nodes
  - theory
    - IT security theory + all child nodes 
    - theory of task technology fit
    - personality theory + all child nodes
    - uncanny valley theory


We also stored the list above in an Excel list for further processing

In [302]:
data_path_group = data_path[:-10]+"Topic 1/Data_Team1/" # create new data path to access files created by Team1

entities_to_removed = pd.read_excel(data_path_group+"ent_to_remove_from_ontology.xlsx") #load excel list as dataframe
entities_to_removed["entity_id_to_remove"] = entities_to_removed.entity_id_to_remove.str.strip() #remove whitespace

In [303]:
entities_to_removed.head(2)

,level_1,level_2,entity_id_to_remove,is_parent_node
0,domain specific entity,IS topic,business process outsourcing,1.0
1,domain specific entity,IS topic,cybersecurity behavior,1.0


In [304]:
# Create list of entites which should be removed
ent_to_rmv_list = entities_to_removed.entity_id_to_remove.to_list()

Now we want to enhance this dataframe to also include: 
- a list of different entities which belong to the specific parent node
- a list of sentences in which the entity occurs
- a list of articles in which the entity occurs
- also we want the number of occurences for all three points from above

In [305]:
tmp_list=[] # List of entities and child nodes which have to be removed from train ontology 
entities_to_remove_cmpl=[] # temporary list to add list of child nodes to each entity

for entity in ent_to_rmv_list:
  
  #print(entity, (entities.values == entity).any(1).astype(int).max()) #to visually check if the random picked nodes are even included in entities database (so if there has been a match yet)
  
  tmp_ = entities[entities.values == entity][["ent_id", "article_id", "sentence_id"]].apply(lambda x: x.unique().tolist()) #get all the unique values for all three columns
  df_tmp = pd.DataFrame(tmp_).T #store results as df
  df_tmp.columns=["ent_list", "articles_list", "sentences_list"] #rename columns
  tmp_list.append(df_tmp) #add df to list

  entities_to_remove_cmpl+=tmp_["ent_id"] # add all parent (and child) entities to a list which then can be excluded from the train dataframe

# Add list of children per entity, list of articles and sentences as new column to dataframe
entities_to_removed = pd.concat((entities_to_removed, pd.concat(tmp_list).reset_index(drop=True)), axis=1)

# add number of occurences as separate columns
entities_to_removed[["num_ent", "num_articles", "num_sentences"]] = entities_to_removed[["ent_list", "articles_list", "sentences_list"]].applymap(lambda x: len(x))

In [306]:
entities_to_removed.sample(5)

,level_1,level_2,entity_id_to_remove,is_parent_node,ent_list,articles_list,sentences_list,num_ent,num_articles,num_sentences
22,methodological entity,data analysis method,sentiment analysis,1.0,"[sentiment analysis, aspect-based sentiment an...","[1417, 4062, 2271, 2661, 443, 5616, 5776, 6106...","[1417_3537_3568, 4062_9470_9492, 4062_19999_20...",2,107,484
0,domain specific entity,IS topic,business process outsourcing,1.0,"[business process outsourcing, knowledge proce...","[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","[5414_1618_1639, 5218_4403_4488, 440_8401_8438...",3,130,351
31,theoretical entity,level of analysis,micro level,1.0,"[individual level, micro level, group level]","[4244, 4245, 3024, 3027, 5413, 5414, 3197, 319...","[4244_11296_11362, 4245_8613_8628, 3024_633_65...",3,1369,4655
9,domain specific entity,Economic sector,structural engineering,1.0,"[sustainable building, structural engineering]","[2470, 1417, 679, 3059, 6172, 1270, 240, 242, ...","[2470_7901_7930, 1417_791_827, 679_3131_3142, ...",2,26,37
4,domain specific entity,IS technology,online marketing,1.0,"[search engine marketing, online advertising, ...","[191, 192, 431, 5414, 4061, 4062, 1623, 2465, ...","[191_2_16, 191_21_55, 191_124_138, 191_138_171...",9,628,5965


In [307]:
# Check number of entities which need to be removed
len(entities_to_remove_cmpl)

121

### Create train dataframe of ontology

In [222]:
# Create train dataframe which does NOT contain any of the entities and porential children
entities_train = entities[~entities.ent_id.isin(entities_to_remove_cmpl)]
entities_train.shape

(1745546, 13)

### Create validation dataframe of ontology

For the validation and test set we need to further split the `entities_to_removed` dataframe

In [ ]:
entities[entities.ent_id.isin(entities_to_remove_cmpl)]

In [145]:
52194/1745546

0.02990124579930864

In [301]:
entities_to_removed.head()

,level_1,level_2,entity_id_to_remove,is_parent_node,article_id,sentence_id,ent_id,num_ent,num_articles,num_sentences,ent_list,articles_list,sentences_list
0,domain specific entity,IS topic,business process outsourcing,1.0,"[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","[5414_1618_1639, 5218_4403_4488, 440_8401_8438...","[business process outsourcing, knowledge proce...",3,130,351,"[business process outsourcing, knowledge proce...","[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","[5414_1618_1639, 5218_4403_4488, 440_8401_8438..."
1,domain specific entity,IS topic,cybersecurity behavior,1.0,"[1622, 1018, 3375, 2474, 6330, 5947, 1845, 425...","[1622_3154_3221, 1622_3221_3320, 1622_3346_336...","[privacy behavior, information security policy...",4,121,695,"[privacy behavior, information security policy...","[1622, 1018, 3375, 2474, 6330, 5947, 1845, 425...","[1622_3154_3221, 1622_3221_3320, 1622_3346_336..."
2,domain specific entity,IS topic,user interface,1.0,"[1836, 434, 4244, 3373, 5413, 5415, 3196, 5606...","[1836_5404_5485, 434_1460_1510, 4244_8878_8906...","[user interface, graphical user interface, voi...",5,980,2694,"[user interface, graphical user interface, voi...","[1836, 434, 4244, 3373, 5413, 5415, 3196, 5606...","[1836_5404_5485, 434_1460_1510, 4244_8878_8906..."
3,domain specific entity,IS topic,IT planning,1.0,"[1834, 4246, 3025, 2467, 4631, 2848, 2851, 369...","[1834_10730_10748, 1834_10941_10959, 4246_7037...","[IT planning, IT policy, IT strategic planning...",4,728,3431,"[IT planning, IT policy, IT strategic planning...","[1834, 4246, 3025, 2467, 4631, 2848, 2851, 369...","[1834_10730_10748, 1834_10941_10959, 4246_7037..."
4,domain specific entity,IS technology,online marketing,1.0,"[191, 192, 431, 5414, 4061, 4062, 1623, 2465, ...","[191_2_16, 191_21_55, 191_124_138, 191_138_171...","[search engine marketing, online advertising, ...",9,628,5965,"[search engine marketing, online advertising, ...","[191, 192, 431, 5414, 4061, 4062, 1623, 2465, ...","[191_2_16, 191_21_55, 191_124_138, 191_138_171..."
